In [1]:
from lark import Lark

In [27]:
grammar = r"""
?start: expr

?expr: sum

?sum: product
    | sum "+" product   -> add
    | sum "-" product   -> sub

?product: power
        | product "\\cdot" power            -> mul
        | product "/" power            -> div
        | product BINOP_FUNC power     -> binfunc_op   // например: a \min b

?power: postfix
      | postfix "^" power              -> pow

?postfix: primary
        | postfix "_" primary          -> subscript

?primary: NUMBER                         -> number
        | GREEK                          -> greek
        | LATIN                          -> latin
        | FRAC frac_expr                 -> frac       // дробь \frac{числитель}{знаменатель}
        | (BINOP_FUNC | FUNCTION) group  -> function_call  // вызов функции, например, "\min{...}" или "\sin{...}"
        | "(" expr ")"                   -> parens
        | "[" expr "]"                   -> brackets
        | "{" expr "}"                   -> braces
        | INTEGRAL integral_limits? expr? -> integral_expr
        | LIMIT limit_limits? expr       -> limit_expr

?group: "{" expr "}" | expr

frac_expr: "{" expr "}" "{" expr "}"

integral_limits: ("_" group)? ("^" group)?
limit_limits: ("_" group)?

// Терминалы для специальных операторов (используются как бинарные операторы, если между двумя выражениями)
BINOP_FUNC: /\\(min|max|opt)/

// Терминалы для функций (если требуется вызов с группой)
FUNCTION: /\\(sin|cos|tan|log|ln|exp)/
FRAC: /\\frac/
NUMBER: /\d+(\.\d+)?/
GREEK: /\\(alpha|beta|gamma|delta|epsilon|zeta|eta|theta|iota|kappa|lambda|mu|nu|xi|omicron|pi|rho|sigma|tau|upsilon|phi|chi|psi|omega)/
LATIN: /[a-zA-Z]+/

INTEGRAL: /\\int/
LIMIT: /\\lim/

%import common.WS
%ignore WS
"""


In [28]:
parser = Lark(grammar, debug=True)
tree = parser.parse(r"a+b^\alpha + \int_a^b x + \ln (b+e) + n \cdot u + \frac {a}{b}")
print(tree.pretty())

add
  add
    add
      add
        latin	a
        pow
          latin	b
          greek	\alpha
      integral_expr
        \int
        integral_limits
          latin	a
          latin	b
        latin	x
    function_call
      \ln
      add
        parens
          add
            latin	b
            latin	e
        mul
          latin	n
          latin	u
  frac
    \frac
    frac_expr
      latin	a
      latin	b

